In [1]:
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from keras.applications.xception import Xception
import DatasetParser
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
wandb: WARNING Keras version 2.3.1 is not fully supported. Required keras >= 2.4.0


In [2]:
(x_train, y_train_raw), (x_test, y_test_raw), class_names = DatasetParser.load_data("Xception")

299
Entrenamiento: 3348
Test: 1440


In [3]:
# One hot encode ouput
y_train = keras.utils.to_categorical(y_train_raw)
y_test = keras.utils.to_categorical(y_test_raw)

In [4]:
Xception_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=True)

In [5]:
from keras.preprocessing import image
import numpy as np
img = image.load_img('elephant.jpg', target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = Xception_model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n03775546', 'mixing_bowl', 0.9016949), ('n02112350', 'keeshond', 0.09713113), ('n03942813', 'ping-pong_ball', 0.0011666244)]


In [6]:
# We should preprocess the images the same way resnet images were preprocessed
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

In [7]:
# Build a new model that is ResNet50 minus the very last layer
last_layer = Xception_model.get_layer("avg_pool")

Xception_layers = keras.Model(inputs=Xception_model.inputs, outputs=last_layer.output)
Xception_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [8]:
# We use our resnet to "predict" but because we have removed the top layer, 
# this outputs the activations of the second to last layer on our dataset

x_train_features = Xception_layers.predict(x_train_preprocessed)

In [9]:
x_test_features = Xception_layers.predict(x_test_preprocessed)

In [10]:
# We can directly stich the models together

Fine_Tuning_model=Sequential()
Fine_Tuning_model.add(Xception_layers)
Fine_Tuning_model.add(Dense(36, activation="softmax"))

Fine_Tuning_model.layers[0].trainable=False

Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Fine_Tuning_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              20861480  
_________________________________________________________________
dense_1 (Dense)              (None, 36)                73764     
Total params: 20,935,244
Trainable params: 73,764
Non-trainable params: 20,861,480
_________________________________________________________________


In [11]:
wandb.init(project='test', entity='scueroinc')
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=10, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

wandb: Currently logged in as: scueroinc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.25 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train on 3348 samples, validate on 1440 samples
Epoch 1/10
3348/3348 [==============================] - 90s 27ms/step - loss: 2.5208 - accuracy: 0.4053 - val_loss: 24.8620 - val_accuracy: 0.0333
Epoch 2/10
3348/3348 [==============================] - 89s 26ms/step - loss: 1.3535 - accuracy: 0.7279 - val_loss: 31.7634 - val_accuracy: 0.0236
Epoch 3/10
3348/3348 [==============================] - 89s 27ms/step - loss: 0.9850 - accuracy: 0.7864 - val_loss: 35.5145 - val_accuracy: 0.0333
Epoch 4/10
3348/3348 [==============================] - 89s 27ms/step - loss: 0.7895 - accuracy: 0.8321 - val_loss: 37.1394 - val_accuracy: 0.0229
Epoch 5/10
3348/3348 [==============================] - 89s 26ms/step - loss: 0.6765 - accuracy: 0.8548 - val_loss: 43.6541 - val_accuracy: 0.0285
Epoch 6/10
3348/3348 [==============================] - 89s 26ms/step - loss: 0.5760 - accuracy: 0.8772 - val_loss: 49.4342 - val_accuracy: 0.0257
Epoch 7/10
3348/3348 [==============================] - 89s 27ms/step 

In [12]:
# We can allow some of the resnet layers to change as we train.  
# Typically you would want to lower the learning rate in conjunction with this.

Fine_Tuning_model.layers[0].trainable = True

# We let the last 3 blocks train
for layer in Fine_Tuning_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in Fine_Tuning_model.layers[0].layers[-11:]:
    layer.trainable = True
    
Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=30, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

Train on 3348 samples, validate on 1440 samples
Epoch 1/30
3348/3348 [==============================] - 100s 30ms/step - loss: 0.5257 - accuracy: 0.8483 - val_loss: 0.9039 - val_accuracy: 0.7840
Epoch 2/30
3348/3348 [==============================] - 96s 29ms/step - loss: 0.1082 - accuracy: 0.9668 - val_loss: 0.8405 - val_accuracy: 0.8007
Epoch 3/30
3348/3348 [==============================] - 97s 29ms/step - loss: 0.0683 - accuracy: 0.9800 - val_loss: 1.0493 - val_accuracy: 0.7736
Epoch 4/30
3348/3348 [==============================] - 97s 29ms/step - loss: 0.0557 - accuracy: 0.9824 - val_loss: 0.8523 - val_accuracy: 0.7875
Epoch 5/30
3348/3348 [==============================] - 97s 29ms/step - loss: 0.0411 - accuracy: 0.9892 - val_loss: 0.7560 - val_accuracy: 0.8313
Epoch 6/30
3348/3348 [==============================] - 96s 29ms/step - loss: 0.0164 - accuracy: 0.9955 - val_loss: 0.7048 - val_accuracy: 0.8493
Epoch 7/30
3348/3348 [==============================] - 96s 29ms/step - los